Import necessary libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import Conv2D, Dense, Flatten, GRU, Input, Lambda
from tensorflow.keras.layers import Dropout, LSTM, TimeDistributed, MaxPool2D, Reshape
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics         import accuracy_score

Connect your Google Drive. 
Since the files are quite big, it is much more convinient to upload them in a cloud service.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Read prepared train-test files

In [ ]:
link_to_drive = '/content/drive/MyDrive/DL_Final_Project/' # Link to a folder with the dataset

Initialization of the Image data generators

In [ ]:
batch_size = 256

train_generator = ImageDataGenerator(rotation_range=90, 
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5, 
                                     height_shift_range=0.5,
                                     horizontal_flip=True, 
                                     vertical_flip=True,
                                     validation_split=0.15,
                                     preprocessing_function=preprocess_input) 

test_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train_data_dir = link_to_drive + 'Train'
test_data_dir = link_to_drive

Creation of train, validation and test generators

In [ ]:
traingen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(32, 32),
                                               class_mode='categorical',
                                               subset='training',
                                               batch_size=batch_size, 
                                               shuffle=True,
                                               seed=42)

validgen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(32, 32),
                                               class_mode='categorical',
                                               subset='validation',
                                               batch_size=batch_size,
                                               shuffle=True,
                                               seed=42)

testgen = test_generator.flow_from_directory(test_data_dir,
                                             target_size=(32, 32),
                                             class_mode=None,
                                             classes=['Test'],
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)

Build the model with freezed leyers and last several trainable layers

In [ ]:
def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=2):
 
    conv_base = ResNet50(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
  
    for layer in conv_base.layers[:-fine_tune]:
        layer.trainable = False
    
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(1072, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)

    model = Model(inputs=conv_base.input, outputs=output_layer)

    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

Initialization of some basic parameters to compile and fit the model

In [ ]:
input_shape = (32, 32, 3)
optim_1 = Adam(learning_rate=0.1)
n_classes = 43
n_steps = traingen.samples // batch_size
n_val_steps = validgen.samples // batch_size
n_epochs = 2

model = create_model(input_shape, n_classes, optim_1, fine_tune=2)

Fit the model

In [ ]:
history = model.fit(traingen,
                    batch_size=batch_size,
                    epochs=n_epochs,
                    validation_data=validgen,
                    steps_per_epoch=n_steps,
                    validation_steps=n_val_steps,
                    verbose=1)

Plot the graphs for accuracy and loss of both datasets

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy and Loss')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

plt.savefig('accuracy_loss.png')

Test the trained model with test dataset

In [ ]:
predict_x=model.predict(testgen) 
classes_x=np.argmax(predict_x,axis=1)

labels = y_test['ClassId'].values
print(accuracy_score(labels, classes_x))